In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

# Function to calculate earnings growth (C - Current Earnings Growth)
def calculate_earnings_growth(ticker):
    stock = yf.Ticker(ticker)
    try:
        # Retrieve historical financial data
        financials = stock.financials.T
        earnings_growth = financials['Net Income'].pct_change().iloc[-1] * 100  # Latest YoY growth in net income
        return earnings_growth
    except KeyError:
        return None

# Function to calculate annual earnings growth (A - Annual Earnings Growth)
def calculate_annual_earnings_growth(ticker):
    stock = yf.Ticker(ticker)
    try:
        financials = stock.financials.T
        annual_growth = financials['Net Income'].pct_change().mean() * 100  # Average annual growth
        return annual_growth
    except KeyError:
        return None

# Function to get institutional sponsorship (I - Institutional Sponsorship)
def get_institutional_sponsorship(ticker):
    stock = yf.Ticker(ticker)
    try:
        major_holders = stock.institutional_holders
        if major_holders is not None:
            # Sum institutional holdings
            institutional_ownership = major_holders['Shares'].sum()
            return institutional_ownership
        else:
            return None
    except Exception:
        return None

# Function to calculate ROE (Return on Equity)
def calculate_roe(ticker):
    stock = yf.Ticker(ticker)
    try:
        financials = stock.financials.T
        balance_sheet = stock.balance_sheet.T
        net_income = financials['Net Income'].iloc[-1]
        equity = balance_sheet['Total Stockholder Equity'].iloc[-1]
        roe = (net_income / equity) * 100  # ROE percentage
        return roe
    except KeyError:
        return None

# Function to calculate debt-to-equity ratio
def calculate_debt_to_equity(ticker):
    stock = yf.Ticker(ticker)
    try:
        balance_sheet = stock.balance_sheet.T
        total_debt = balance_sheet.get('Total Debt', None)
        total_equity = balance_sheet.get('Total Stockholder Equity', None)
        if total_debt is not None and total_equity is not None:
            debt_to_equity = total_debt / total_equity
            return debt_to_equity
        else:
            return None
    except KeyError:
        return None

# Function to calculate dividend growth
def calculate_dividend_growth(ticker):
    stock = yf.Ticker(ticker)
    try:
        dividends = stock.dividends
        if not dividends.empty:
            dividend_growth = dividends.pct_change().mean() * 100  # Average YoY dividend growth
            return dividend_growth
        else:
            return None
    except Exception:
        return None

# Function to rank S&P 500 companies based on CANSLIM strategy
def evaluate_canslim_parameters(ticker):
    try:
        earnings_growth = calculate_earnings_growth(ticker)
        annual_earnings_growth = calculate_annual_earnings_growth(ticker)
        institutional_sponsorship = get_institutional_sponsorship(ticker)
        roe = calculate_roe(ticker)
        debt_to_equity = calculate_debt_to_equity(ticker)
        dividend_growth = calculate_dividend_growth(ticker)
        
        print(f"Results for {ticker}:")
        print(f"Earnings Growth (C): {earnings_growth if earnings_growth is not None else 'N/A'}")
        print(f"Annual Earnings Growth (A): {annual_earnings_growth if annual_earnings_growth is not None else 'N/A'}")
        print(f"Institutional Sponsorship (I): {institutional_sponsorship if institutional_sponsorship is not None else 'N/A'}")
        print(f"ROE: {roe if roe is not None else 'N/A'}")
        print(f"Debt-to-Equity: {debt_to_equity if debt_to_equity is not None else 'N/A'}")
        print(f"Dividend Growth: {dividend_growth if dividend_growth is not None else 'N/A'}\n")
        
        return {
            'Ticker': ticker,
            'Earnings Growth': earnings_growth,
            'Annual Growth': annual_earnings_growth,
            'Institutional Sponsorship': institutional_sponsorship,
            'ROE': roe,
            'Debt to Equity': debt_to_equity,
            'Dividend Growth': dividend_growth
        }
    except Exception as e:
        print(f"Error processing {ticker}: {e}")
        return None

# Load the list of S&P 500 companies (You can download this as a CSV or use yfinance)
def get_sp500_companies():
    sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
    return sp500['Symbol'].tolist()

# Evaluate CANSLIM strategy for all S&P 500 companies
def evaluate_sp500_canslim():
    sp500_tickers = get_sp500_companies()
    results = []
    
    for ticker in sp500_tickers[:50]:  # Limit to first 50 tickers for demonstration
        result = evaluate_canslim_parameters(ticker)
        if result:
            results.append(result)
    
    # Convert results to DataFrame
    results_df = pd.DataFrame(results)
    
    # Save the results to CSV
    results_df.to_csv('sp500_canslim_results.csv', index=False)
    print("Results saved to 'sp500_canslim_results.csv'")
    
    return results_df

# Main function
if __name__ == "__main__":
    sp500_results = evaluate_sp500_canslim()
    print(sp500_results.head())
